# SELECT NEARBY NODES 
In this notebook, we make a subselection of the sessionGraph based on each nodes' cartesianTransform & orientedBounds
For every node, we evaluate the Euclidean distance within a threshold to the source.
As ouput, the method generates a selection of nodes 

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [8]:
#IMPORT PACKAGES
from rdflib import Graph

import os.path, time
import importlib
import numpy as np
import open3d as o3d

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

In [9]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload 2

## 1. INITIALIZE SESSION

In [11]:
## INPUTS
projectPath= os.path.abspath(os.path.join(os.getcwd(), os.pardir))+"\\tests"#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = projectPath + "\\Samples3" #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
t_d =10.0 # (float) search distance
t_x=2.0 # (float) search distance in X given the boundingbox
t_y=2.0 # (float) search distance in Y given the boundingbox
t_z=1.0 # (float) search distance in Z given the boundingbox

idx=2 # index of the node you want to test

## 2. READ PRESENT GRAPHS
A specifc NodeType (Mesh,PCD,IMG,BIM) is created per resource in the session

In [12]:
## read all graph files in a session and merge them
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
rdfGraphPaths=[]

# collect all RDF graphs in the session
sessionGraph = Graph()
for path in allSessionFilePaths:        
    if path.endswith(".ttl") and 'features' not in path:
        rdfGraphPaths.append(path)
        sessionGraph.parse(path)     
print('Total graph size = '+str(len(sessionGraph)) +' triples from '+ str(len(rdfGraphPaths))+' paths')

Total graph size = 13691 triples from 2 paths


# 3. A Node is created per subject in the joint RDF Graph

In [13]:
nodelist=tl.graph_to_nodes(sessionGraph,sessionPath=sessionPath)
print (str(len(nodelist))+' Nodes succesfully created from the session!')
node=nodelist[idx]

971 Nodes succesfully created from the session!


# WRAPPED SELECTIONS

In [14]:
selectedNodes1=tl.select_k_nearest_nodes(node,nodelist,k=1)
selectedNodes2=tl.select_nodes_with_centers_in_radius(node,nodelist,r=20)
selectedNodes3=tl.select_nodes_with_centers_in_bounding_box(node,nodelist,u=1,v=1,w=0.5)
selectedNodes4=tl.select_nodes_with_bounding_points_in_bounding_box(node,nodelist,u=1,v=1,w=0.5)
selectedNodes5=tl.select_nodes_with_intersecting_bounding_box(node,nodelist,u=1,v=1,w=0.5)
selectedNodes6=tl.select_nodes_with_intersecting_meshes(node,nodelist)

print('selectedNodes1: '+str(len(selectedNodes1)))
print('selectedNodes2: '+str(len(selectedNodes2)))
print('selectedNodes3: '+str(len(selectedNodes3)))
print('selectedNodes4: '+str(len(selectedNodes4)))
print('selectedNodes5: '+str(len(selectedNodes5)))
print('selectedNodes6: '+str(len(selectedNodes6)))


No file containing this object's name and extension is found in the sessionfolder


TypeError: read_triangle_mesh(): incompatible function arguments. The following argument types are supported:
    1. (filename: str, enable_post_processing: bool = False, print_progress: bool = False) -> open3d.cpu.pybind.geometry.TriangleMesh

Invoked with: None

## SELECTION 1
select nodes within t_d

In [ ]:
#create pcd from cartesianTransforms or cartesianBounds
pcd = o3d.geometry.PointCloud()
array=np.empty((1,3),dtype=float)
for node in nodelist:
    if getattr(node,'cartesianTransform',None) is not None: 
        center=gmu.get_translation(node.cartesianTransform) 
        array=np.vstack([array,center])
    elif getattr(node,'cartesianBounds',None) is not None:
        center=gmu.get_center_of_cartesian_bounds(node.cartesianBounds) 
        array=np.vstack([array,center])
pcd.points = o3d.utility.Vector3dVector(array)

#Create KDTree from pcd
pcdTree = o3d.geometry.KDTreeFlann(pcd)

#Find its 200 nearest neighbors")
[k1, indices1, _] = pcdTree.search_knn_vector_3d(pcd.points[200], 200)

#Find its neighbors with distance less than 0.2
[k2, indices2, _] = pcdTree.search_radius_vector_3d(pcd.points[0], 0.2)
print(str(indices1))
print(str(indices2))


IntVector[200, 466, 686, 788, 209, 264, 263, 799, 242, 243, 886, 635, 915, 780, 888, 269, 673, 634, 889, 124, 21, 672, 930, 445, 855, 719, 459, 633, 54, 914, 793, 241, 674, 240, 965, 310, 814, 518, 590, 508, 608, 694, 671, 22, 589, 853, 935, 713, 588, 715, 931, 783, 708, 657, 446, 450, 507, 151, 757, 443, 509, 815, 100, 781, 318, 837, 772, 854, 838, 882, 933, 710, 964, 718, 963, 675, 510, 891, 932, 439, 709, 620, 448, 810, 109, 918, 948, 890, 949, 693, 506, 809, 637, 950, 677, 591, 670, 631, 511, 11, 835, 636, 244, 201, 512, 377, 211, 386, 856, 720, 881, 540, 592, 449, 539, 522, 224, 387, 756, 12, 505, 685, 808, 378, 945, 616, 586, 541, 652, 538, 839, 789, 753, 885, 947, 98, 53, 222, 123, 542, 946, 800, 707, 564, 916, 775, 841, 563, 692, 836, 2, 609, 23, 617, 682, 444, 447, 50, 99, 829, 740, 831, 289, 782, 669, 168, 58, 618, 219, 834, 307, 565, 232, 108, 255, 893, 535, 857, 239, 892, 562, 458, 227, 218, 77, 57, 441, 630, 379, 290, 388, 784, 13, 265, 629, 394, 566, 97, 668, 752]
IntVect

In [ ]:
print(pcd.points[0])

[86.96807314 66.24003458 47.85120527]


In [ ]:
pcd.paint_uniform_color([1,0,0])
# a = o3d.geometry.TriangleMesh.create_sphere(radius=10)
# a.translate([-11.96593745 , 95.40430626 ,  8.92250137])
a = o3d.geometry.TriangleMesh.create_sphere(radius=10)
a.translate([-3.96593745 , 95.40430626 ,  8.92250137])
box=a.get_oriented_bounding_box()
box.color=[0,0,1]
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]
o3d.visualization.draw_geometries([pcd,box])

# 5. SELECTION 2
Select nodes of which the centers lie within bounding box of the source node with an offset [u,v,z]

In [ ]:
centers=np.empty((len(nodelist),3),dtype=float)
boxes=np.empty((len(nodelist),1),dtype=o3d.geometry.OrientedBoundingBox)
# pcds=np.empty((len(nodelist),1),dtype=o3d.geometry.PointCloud)

for idx,node in enumerate(nodelist):
    if getattr(node,'orientedBounds',None) is not None:
        centers[idx,:]=node.get_center() 
        box=gmu.oriented_bounds_to_open3d_oriented_bounding_box(node.orientedBounds)      
        box=gmu.expand_box(box,u=t_x,v=t_y,w=t_z) # to be tested
        box.color=[1, 0, 0]
        boxes[idx] = box

#points are the centers of all the nodes
pcd = o3d.geometry.PointCloud()
points = o3d.utility.Vector3dVector(centers)
pcd.points=points

# #Find the nodes that lie within the index box 
idx=boxes[idx,0].get_point_indices_within_bounding_box(points)
print(str(idx))

[0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 18, 19, 20, 21, 22, 23, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 52, 53, 56, 57, 59, 60, 61, 62, 63, 66, 67, 68, 69, 70, 73, 74, 75, 76, 77, 80, 81, 82, 83, 84, 85, 87, 89, 90, 91, 92, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 128, 129, 130, 131, 133, 134, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 183, 184, 191, 192, 194, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 210, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 230, 231, 232, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 285

In [ ]:
#visualize the result
colors=np.zeros((len(nodelist),3))
# pcd.paint_uniform_color([0, 0.651, 0.929])
for i,color in enumerate(colors):
    colors[idx,:]=[0.0, 0.0, 1.0]
    if i==idx:
        colors[idx,:]=[1.0,0.0,0.0]

pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([pcd,boxes[idx,0]])

# 6. SELECTION 3
Select nodes of which the verices of the bounding boxes intersect

In [31]:
centers=np.empty((len(nodelist),3),dtype=float)
boxes=np.empty((len(nodelist),1),dtype=o3d.geometry.OrientedBoundingBox)

for idx,node in enumerate(nodelist):
    if getattr(node,'orientedBounds',None) is not None:
        centers[idx,:]=node.get_center()  
        box=gmu.oriented_bounds_to_open3d_oriented_bounding_box(node.orientedBounds)      
        box=gmu.expand_box(box,u=t_x,v=t_y,w=t_z) # to be tested
        box.color=[1, 0, 0]
        boxes[idx] = box

result=gmu.get_box_inliers(boxes[idx,0],boxes[:,0])
print(str(result))

[2, 3, 4, 7, 13, 15, 24, 25, 26, 28, 30, 32, 35, 36, 38, 39, 42, 45, 46, 51, 52, 53, 55, 56, 57, 58, 59, 60, 63, 67, 68, 69, 71, 74, 77, 80, 81, 82, 83, 85, 86, 88, 89, 90, 91, 94, 95, 96, 107, 110, 115, 121, 125, 126, 128, 131, 132, 138, 139, 145, 146, 148, 149, 151, 152, 154, 155, 156, 158, 160, 165, 171, 172, 173, 175, 177, 179, 181, 188, 192, 201, 203, 206, 207, 212, 220, 221, 224, 225, 233, 234, 238, 242, 244, 245, 248, 250, 253, 256, 258, 259, 265, 268, 270, 271, 272, 273, 274, 275, 276, 277, 279, 280, 282, 283, 284, 285, 288, 289, 290, 291, 295, 298, 299, 300, 301, 305, 308, 310, 311, 312, 315, 316, 320, 322, 328, 330, 331, 332, 334, 336, 338, 341, 343, 345, 346, 349, 353, 354, 356, 358, 360, 363, 365, 366, 371, 374, 377, 379, 380, 381, 388, 389, 390, 391, 393, 395, 396, 397, 398, 399, 400, 402, 403, 407, 408, 410, 411, 412, 415, 421, 424, 426, 427, 428, 430, 432, 434, 435, 438, 439, 440, 443, 445, 451, 452, 453, 457, 460, 462, 464, 465, 466, 468, 470, 472, 473, 476, 478, 482, 4

In [ ]:
#visualise results
list=[]
for i,box in enumerate(boxes[:,0]):
    if box !=None and len(result) !=0:
        box.color=[1, 0, 0]
        if i in result:
            box.color=[0, 0, 1]
        list.append(box)
o3d.visualization.draw_geometries(list)

# 7. SELECTION 4
Select nodes of which the geometry of the bounding boxes intersect

In [33]:
centers=np.empty((len(nodelist),3),dtype=float)
boxes=np.empty((len(nodelist),1),dtype=o3d.geometry.OrientedBoundingBox)

for idx,node in enumerate(nodelist):
    if getattr(node,'orientedBounds',None) is not None:
        centers[idx,:]=node.get_center()  
        box=gmu.oriented_bounds_to_open3d_oriented_bounding_box(node.orientedBounds)      
        box=gmu.expand_box(box,u=t_x,v=t_y,w=t_z) # to be tested
        box.color=[1, 0, 0]
        boxes[idx] = box


result=gmu.get_box_intersections(boxes[idx,0],boxes[:,0]) #to check
print(str(result))

[0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 56, 57, 59, 60, 61, 62, 63, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 80, 81, 82, 83, 84, 85, 87, 89, 90, 91, 92, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 128, 129, 130, 131, 133, 134, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 187, 188, 191, 192, 194, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 210, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 230, 231, 232, 233, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 268, 269, 270, 271, 2

In [ ]:
#visualise results
list=[]
for i,box in enumerate(boxes[:,0]):
    if box !=None and len(result) !=0:
        box.color=[1, 0, 0]
        if i in result:
            box.color=[0, 0, 1]
        list.append(box)
list[0].color=[0,1,0]
o3d.visualization.draw_geometries(list)